In [ ]:
"""
T_PROCESSUS = STRUCTURE
   nom      : CHAINE
   arrivee  : ENTIER
   duree    : ENTIER
   reste    : ENTIER
   debut    : ENTIER
   fin      : ENTIER
   sejour   : ENTIER
   attente  : ENTIER
   reponse  : ENTIER
   traite   : BOOLEEN
   dansFile : BOOLEEN
FINSTRUCTURE

T_TABPROC = TABLEAU[1..100] DE T_PROCESSUS

T_FILE = STRUCTURE
   tab   : TABLEAU[1..100] DE ENTIER
   tete  : ENTIER
   queue : ENTIER
   nb    : ENTIER
FINSTRUCTURE

"""


"""
ALGORITHME RoundRobin_FileFIFO

CONSTANTES
   MAX : ENTIER ← 100

VARIABLES
   proc         : T_TABPROC
   file         : T_FILE
   n            : ENTIER         { nombre de processus }
   q            : ENTIER         { quantum }
   i            : ENTIER
   tempsCourant : ENTIER
   nbTermines   : ENTIER
   prochainArr  : ENTIER
   p            : ENTIER         { indice du processus courant }
   exec         : ENTIER         { temps d'exécution pour un tour }

DEBUT

   { 1. Lecture des données }

   ECRIRE("Donner le nombre de processus (<= ", MAX, ") :")
   LIRE(n)

   ECRIRE("Donner la valeur du quantum :")
   LIRE(q)

   POUR i DE 1 A n FAIRE

      ECRIRE("Nom du processus ", i, " :")
      LIRE(proc[i].nom)

      ECRIRE("Temps d'arrivee de ", proc[i].nom, " :")
      LIRE(proc[i].arrivee)

      ECRIRE("Temps d'execution (duree) de ", proc[i].nom, " :")
      LIRE(proc[i].duree)

      proc[i].reste    ← proc[i].duree
      proc[i].debut    ← -1
      proc[i].fin      ← 0
      proc[i].sejour   ← 0
      proc[i].attente  ← 0
      proc[i].reponse  ← 0
      proc[i].traite   ← FAUX
      proc[i].dansFile ← FAUX

   FINPOUR

   { 2. Initialisation de la file }

   file.nb    ← 0
   file.tete  ← 1
   file.queue ← 0

   { 3. Initialisation du temps et du compteur }

   tempsCourant ← 0
   nbTermines   ← 0

   { 4. Fonctionnalité interne : ajouter les nouveaux processus prêts dans la file
        (arrives et non termines, et pas encore dans la file) }

   { On va coder ça "inline" : une boucle qu'on réutilise partout }

   { 4.1 Première insertion : processus arrivés au temps 0 }

   POUR i DE 1 A n FAIRE
      SI (proc[i].traite = FAUX) ET (proc[i].dansFile = FAUX) ET (proc[i].arrivee <= tempsCourant) ALORS

         { ENFILER i }
         SI file.nb < MAX ALORS
            file.queue ← file.queue + 1
            SI file.queue > MAX ALORS
               file.queue ← 1
            FINSI
            file.tab[file.queue] ← i
            file.nb ← file.nb + 1
            proc[i].dansFile ← VRAI
         FINSI

      FINSI
   FINPOUR

   { 5. Boucle principale Round Robin }

   TANTQUE nbTermines < n FAIRE

      { 5.1 Si la file est vide, avancer le temps au prochain arrivee }

      SI file.nb = 0 ALORS

         prochainArr ← -1

         POUR i DE 1 A n FAIRE
            SI proc[i].traite = FAUX ALORS
               SI (prochainArr = -1) OU (proc[i].arrivee < prochainArr) ALORS
                  prochainArr ← proc[i].arrivee
               FINSI
            FINSI
         FINPOUR

         tempsCourant ← prochainArr

         { Ajouter les processus arrives à ce moment }

         POUR i DE 1 A n FAIRE
            SI (proc[i].traite = FAUX) ET (proc[i].dansFile = FAUX) ET (proc[i].arrivee <= tempsCourant) ALORS

               { ENFILER i }
               SI file.nb < MAX ALORS
                  file.queue ← file.queue + 1
                  SI file.queue > MAX ALORS
                     file.queue ← 1
                  FINSI
                  file.tab[file.queue] ← i
                  file.nb ← file.nb + 1
                  proc[i].dansFile ← VRAI
               FINSI

            FINSI
         FINPOUR

      SINON   { file.nb > 0 }

         { 5.2 DEFILER le processus en tête de file }

         p ← file.tab[file.tete]
         file.tete ← file.tete + 1
         SI file.tete > MAX ALORS
            file.tete ← 1
         FINSI
         file.nb ← file.nb - 1
         proc[p].dansFile ← FAUX

         { 5.3 Si c'est la première fois qu'il passe au processeur, fixer son debut }

         SI proc[p].debut = -1 ALORS
            proc[p].debut ← tempsCourant
         FINSI

         { 5.4 Déterminer le temps d'exécution pour ce tour }

         exec ← q
         SI proc[p].reste < exec ALORS
            exec ← proc[p].reste
         FINSI

         { 5.5 Exécution pendant 'exec' unités de temps }

         tempsCourant   ← tempsCourant + exec
         proc[p].reste  ← proc[p].reste - exec

         { 5.6 Ajouter les nouveaux processus arrives pendant ce temps }

         POUR i DE 1 A n FAIRE
            SI (proc[i].traite = FAUX) ET (proc[i].dansFile = FAUX) ET (proc[i].arrivee <= tempsCourant) ALORS

               { ENFILER i }
               SI file.nb < MAX ALORS
                  file.queue ← file.queue + 1
                  SI file.queue > MAX ALORS
                     file.queue ← 1
                  FINSI
                  file.tab[file.queue] ← i
                  file.nb ← file.nb + 1
                  proc[i].dansFile ← VRAI
               FINSI

            FINSI
         FINPOUR

         { 5.7 Vérifier si le processus p est terminé }

         SI proc[p].reste = 0 ALORS

            proc[p].fin     ← tempsCourant
            proc[p].sejour  ← proc[p].fin - proc[p].arrivee
            proc[p].reponse ← proc[p].debut - proc[p].arrivee
            proc[p].attente ← proc[p].sejour - proc[p].duree

            proc[p].traite  ← VRAI
            nbTermines      ← nbTermines + 1

         SINON

            { Le processus n'est pas terminé : on le remet en fin de file }

            SI file.nb < MAX ALORS
               file.queue ← file.queue + 1
               SI file.queue > MAX ALORS
                  file.queue ← 1
               FINSI
               file.tab[file.queue] ← p
               file.nb ← file.nb + 1
               proc[p].dansFile ← VRAI
            FINSI

         FINSI

      FINSI

   FINTANTQUE

   { 6. Affichage des résultats }

   ECRIRE("Resultats de l'ordonnancement Round Robin :")
   POUR i DE 1 A n FAIRE

      ECRIRE("Processus : ", proc[i].nom)
      ECRIRE("  Temps d'arrivee    : ", proc[i].arrivee)
      ECRIRE("  Duree (execution)  : ", proc[i].duree)
      ECRIRE("  Debut execution    : ", proc[i].debut)
      ECRIRE("  Fin execution      : ", proc[i].fin)
      ECRIRE("  Temps de sejour    : ", proc[i].sejour)
      ECRIRE("  Temps d'attente    : ", proc[i].attente)
      ECRIRE("  Temps de reponse   : ", proc[i].reponse)

   FINPOUR

FIN

"""